<a href="https://colab.research.google.com/github/arumshin-dev/python_conda_jupyter/blob/main/codeit/3_3_2_ALBERT_%E1%84%89%E1%85%A1%E1%84%8C%E1%85%A5%E1%86%AB%E1%84%92%E1%85%A1%E1%86%A8%E1%84%89%E1%85%B3%E1%86%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ALBERT 모델

기본 적으로 BERT 모델은 인코딩 블록이 깊어질 수 록 파라미터의 규모가 매우 커질 뿐만 아니라, 매우 큰 어휘 사전(vocab)과 임베딩 길이를 사용할 때, 임베딩 매트릭스가 차지하는 파라미터가 너무 커져 버리는 문제가 생깁니다.

ALBERT는 크게 2가지 기법을 활용하여 이를 크게 낮추고 모델 파라미터 규모를 효과적으로 줄입니다.


- **임베딩 파라미터의 분리(Factorized Embedding Parameterization)**

    * (단어 임베딩 차원) → (프로젝션 레이어) → (Transformer hidden 차원)으로 분리하여 임베딩 행렬의 파라미터를 크게 감소

    * 단어 임베딩은 작은 차원(예: 128차원)으로 두고, 이후 선형 레이어를 통해 Transformer 블록에서 사용하는 숨김(hidden) 차원(예: 256~768차원 등)으로 매핑

- **크로스-레이어 파라미터 공유(Cross-Layer Parameter Sharing)**

    * 여러 개의 Transformer 레이어가 모두 같은 가중치를 공유하여, 한 개의 Transformer 블록을 N번 반복해서 호출(레이어를 복사해서 쓰는 것이 아닌, ‘같은’ 가중치를 사용하는 구조)

또한, ALBERT는 BERT의 NSP 사전학습 대신 두 문장의 순서가 올바른지를 맞히는 SOP 사전학습을 진행합니다. SOP는 NSP의 랜덤 방식에 비해 언어적 일관성과 문맥 연결성을 더 효과적으로 학습합니다.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
import torch
from torch import nn
import copy
from torch.utils.data import Dataset, DataLoader, random_split
import sentencepiece as spm
import pandas as pd
import numpy as np
import math

In [ ]:
df = pd.read_csv(f'./train.csv')
df[['HS01','SS01']]

,HS01,SS01
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...
...,...,...
51623,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.,경제적인 문제 때문에 막막하시군요. 마음이 편치 않으시겠어요.
51624,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.,건강에 대한 어려움 때문에 기분이 좋지 않으시군요. 속상하시겠어요.
51625,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.,노후 준비에 대한 어려움 때문에 걱정이 많으시겠어요.
51626,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.,가족과의 문제 때문에 속상하시겠어요.


### 학습 데이터세트 구성

In [ ]:
# BPE :바이트 페어 인코딩(Byte Pair Encoding, BPE)
import os, re

# 추가 쓰기모드로 텍스트 파일 열기
with open('train.txt', 'w', encoding='utf-8') as f:
  for text in df['HS01']:
        text = str(text)
        text = re.sub(r'[^\w\s]', '', text)     # 특수문자 제거
        text = re.sub(r'[\n\t]', ' ', text)     # 줄바꿈, 탭 제거
        text = re.sub(r'\s+', ' ', text)        # 연속된 공백 제거
        text= text.strip()                      # 문장 양끝 공백 제거
        try:
            f.write(text+'\n')
        except:
                pass

# 저장 경로 생성
os.makedirs('./bpe', exist_ok=True)

spm.SentencePieceTrainer.train(
    input='train.txt',                      # 텍스트 뭉치 파일
    model_prefix='./bpe/spm_krsent',        # 출력 모델 파일 이름
    vocab_size=2000                         # 토큰 개수
)

spm.SentencePieceTrainer.train(input='train.txt',               # 텍스트 뭉치 파일
                            model_prefix='./bpe/spm_krsent',    # 출력 모델 파일 이름
                            vocab_size=4000,                    # 토큰 개수
                            bos_id=1,
                            eos_id=2,
                            unk_id=3,
                            pad_id=0
                            )

In [ ]:
class SPDataSet(Dataset):
    def __init__(self, sp, max_len):
        self.max_len = max_len
        self.df = pd.read_csv(f'./train.csv')
        self.sp = sp

        # SOP 태스크를 위한 (sent1, sent2, label) 리스트
        self.pairs = []

        # "올바른 순서" => label = 0
        # "뒤집힌 순서" => label = 1
        for _, item in self.df.iterrows():
            sent1 = item['HS01']
            sent2 = item['SS01']

            # 원래 순서
            self.pairs.append((sent1, sent2, 0))
            # 뒤집힌 순서
            self.pairs.append((sent2, sent1, 1))

        # 10개의 문장쌍만 확인 (디버그 용도)
        print(self.pairs[:10])

    def zero_pad(self, tok):
        """토큰 리스트를 max_len 길이에 맞춰 제로 패딩"""
        if len(tok) >= self.max_len:
            return tok[:self.max_len]
        else:
            padding = np.zeros(self.max_len, dtype=np.int64)
            padding[:len(tok)] = tok
            return padding

    def __getitem__(self, idx):
        sent1, sent2, label = self.pairs[idx]

        # SentencePiece로 문장 → ID 시퀀스
        sent1_ids = self.sp.encode_as_ids(sent1)
        sent2_ids = self.sp.encode_as_ids(sent2)

        # 두 문장을 [BOS] + sent1 + [EOS] + sent2 + [EOS]로 이어 붙임
        inp = [self.sp.bos_id()] + sent1_ids + [self.sp.eos_id()] \
              + sent2_ids + [self.sp.eos_id()]

        inp = self.zero_pad(inp)  # (max_len,) numpy array
        inp_tensor = torch.tensor(inp, dtype=torch.long)

        # 패딩 마스크
        mask = inp_tensor.eq(0)   # True/False 텐서

        return inp_tensor, torch.tensor(label, dtype=torch.long), mask

    def __len__(self):
        return len(self.pairs)


# 예시 사용
sp = spm.SentencePieceProcessor(model_file=f'./bpe/spm_krsent.model')
dataset = SPDataSet(sp, max_len=60)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

for inp, tar, mask in dataloader:
    print(f'입력 토큰 : {inp}')
    print(f'SOP 라벨 : {tar}')
    print(f'패딩 마스크 : {mask}')
    break

[('일은 왜 해도 해도 끝이 없을까? 화가 난다.', '많이 힘드시겠어요. 주위에 의논할 상대가 있나요?', 0), ('많이 힘드시겠어요. 주위에 의논할 상대가 있나요?', '일은 왜 해도 해도 끝이 없을까? 화가 난다.', 1), ('이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.', '급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?', 0), ('급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?', '이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.', 1), ('회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스트레스 받아. ', '회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 하면 좋을까요?', 0), ('회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 하면 좋을까요?', '회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스트레스 받아. ', 1), ('직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 섭섭해.', '관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있게 도움을 줄까요?', 0), ('관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있게 도움을 줄까요?', '직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 섭섭해.', 1), ('얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.', '무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는 것일까요?', 0), ('무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는 것일까요?', '얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무

In [ ]:
def mask_tokens_for_mlm(input_ids, special_tokens_ids, mask_token_id, vocab_size, mlm_prob=0.15):
    device = input_ids.device
    B, L = input_ids.shape

    # 입력 토큰을 복사하여 사용
    masked_input_ids = input_ids.clone()
    # 라벨 토큰을 위해 동일 형상의 -100 텐서 생성
    mlm_labels = torch.full_like(input_ids, -100)  # -100은 라벨에서 무시되는 토큰으로 사용

    # 스페셜 토큰 위치를 True
    special_mask = torch.zeros_like(input_ids, dtype=torch.bool)
    for sp_id in special_tokens_ids:
        special_mask |= (input_ids == sp_id)

    # 0~1 사이의 무작위 확률 분포 생성
    rand_vals = torch.rand_like(input_ids.float())

    # mlm_prob(0.15) 보다 작고 스페셜 토큰인 아닌경우만
    to_mask = (rand_vals < mlm_prob) & (~special_mask)

    # 마스킹된 위치의 정답 레이블 = 원본 토큰
    mlm_labels[to_mask] = input_ids[to_mask]

    # BERT 논문 비율: 80% -> [MASK], 10% -> random, 10% -> 원본
    mask_choices = torch.rand_like(input_ids.float())

    # mask_choices에서 80%에 해당하고 마스크 인덱스인 경우 마스크 토큰으로
    mask_1 = (mask_choices < 0.8) & to_mask
    masked_input_ids[mask_1] = mask_token_id

    # mask_choices에서 10%에 해당하고 마스크 인덱스인 경우 랜덤 토큰 변환
    mask_2 = (mask_choices >= 0.8) & (mask_choices < 0.9) & to_mask
    random_tokens = torch.randint(0, vocab_size, size=(), device=device)
    masked_input_ids[mask_2] = random_tokens

    return masked_input_ids, mlm_labels

### 모델링

In [ ]:
class MTBlock(nn.Module):
    """
    🏗️ Multi-Head Transformer Block
    - Transformer의 기본 구성 요소
    - Self-Attention + Feed Forward Network
    """
    def __init__(self, hidden_dim, nhead, feed_dim=512, gelu=False, dropout=0.):
        super(MTBlock, self).__init__()

        self.mha = nn.MultiheadAttention(hidden_dim, nhead, dropout=dropout, batch_first=True)
        if gelu:
            self.active = nn.GELU()
        else:
            self.active = nn.ReLU()

        self.ffn = nn.Sequential(
            nn.Linear(hidden_dim, feed_dim),
            self.active,
            nn.Dropout(dropout),
            nn.Linear(feed_dim, hidden_dim)
        )

        self.norm1 = nn.LayerNorm(hidden_dim, eps=1e-6)
        self.norm2 = nn.LayerNorm(hidden_dim, eps=1e-6)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # 셀프 어텐션
        attn_output, _ = self.mha(x, x, x, key_padding_mask=mask, need_weights=False)
        attn_output = self.dropout1(attn_output)
        out1 = self.norm1(x + attn_output)

        # 피드포워드
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        out2 = self.norm2(out1 + ffn_output)

        return out2

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, embed_dim, max_pos=10000):
        super().__init__()
        # 위치 인덱스
        position = torch.arange(max_pos).unsqueeze(1)
        # 위치에 따른 각도 추출
        # 지수(exp)와 로그(log)를 함께써 지수승 형태(**)를 만들어 사용
        div_term = torch.exp(torch.arange(0, embed_dim, 2) * (-math.log(10000.0) / embed_dim))

        # 제로 텐서를 만들고 sin,cos 결과 할당
        pe = torch.zeros(1, max_pos, embed_dim)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe) # 학습되지 않는 모듈로 저장

    def forward(self, x):
        # 임베딩이 입력되면 포지션과 더하여 반환
        x = x + self.pe[:, :x.size(1), :]
        return x

In [ ]:
class SimpleALBERT(nn.Module):
    def __init__(
        self,
        vocab_size,
        embedding_dim=64,
        hidden_dim=256,
        num_heads=4,
        feed_dim=512,
        num_layers=4,
        num_classes=2,   # SOP 또한 이진 분류
        dropout=0.1
    ):
        super().__init__()

        # 작은 차원의 단어 임베딩
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # 임베딩 프로젝션 → hidden_dim
        self.embedding_proj = nn.Linear(embedding_dim, hidden_dim)

        # 위치 인코딩
        self.pos_encoding = PositionalEncoding(hidden_dim)
        self.dropout = nn.Dropout(dropout)

        # ALBERT: 모든 레이어 가중치 공유
        self.transformer_block = MTBlock(hidden_dim, num_heads, feed_dim, dropout=dropout)
        self.num_layers = num_layers

        # MLM 헤드
        self.mlm_head = nn.Linear(hidden_dim, vocab_size)

        # SOP 헤드
        self.sop_head = nn.Linear(hidden_dim, num_classes)

    def forward(self, x, mask=None):
        batch_size, seq_len = x.shape
        device = x.device

        # 임베딩 + 프로젝션
        emb = self.word_embeddings(x)            # (B, L, embedding_dim)
        emb = self.embedding_proj(emb)           # (B, L, hidden_dim)

        # 위치 인코딩 & 드롭아웃
        emb = self.pos_encoding(emb)
        emb = self.dropout(emb)

        # 동일한 Transformer 블록을 num_layers번 반복(가중치 공유)
        x_out = emb
        for _ in range(self.num_layers):
            x_out = self.transformer_block(x_out, mask)

        # MLM 로짓
        mlm_logits = self.mlm_head(x_out)        # (B, L, vocab_size)

        # CLS 토큰(첫 번째 토큰)으로 SOP 로짓
        cls_emb = x_out[:, 0]                    # (B, hidden_dim)
        sop_logits = self.sop_head(cls_emb)      # (B, 2)

        return mlm_logits, sop_logits

vocab_size = sp.get_piece_size()
model = SimpleALBERT(vocab_size=vocab_size)
with torch.no_grad():
    for inp, tar, mask in dataloader:
        mlm_logits, nsp_logits = model(inp.long(), mask)
        print(mlm_logits.shape)
        print(nsp_logits)
        break

torch.Size([2, 60, 4000])
tensor([[-0.3710, -0.0163],
        [-0.0922,  0.0738]])


### 모델 학습

In [ ]:
# 하이퍼파라미터 정의
seq_len = 100
embed_dim = 64
hidden_dim=128
num_heads = 4
feed_dim = 256
num_layers = 4
num_classes = 2
num_epochs = 50
batch_size = 64
lr = 1e-4

# MLM 변환 함수에 넣어줄 스페셜 토큰 정의
mask_id = sp.get_piece_size()
special_tokens_ids = [sp.bos_id(), sp.eos_id(), 0]

# 마스크 토큰이 추가되므로 토큰개수에 +1
sp = spm.SentencePieceProcessor(model_file=f'./bpe/spm_krsent.model')
vocab_size = sp.get_piece_size() + 1

# 데이터세트 분할
dataset = SPDataSet(sp, seq_len)
generator1 = torch.Generator().manual_seed(42)
test_dataset, train_dataset = random_split(dataset, [0.2, 0.8], generator=generator1)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f'train dataset size: {len(train_dataset)}')
print(f'test dataset size: {len(test_dataset)}')

# 모델 생성
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleALBERT(vocab_size, embed_dim, hidden_dim, num_heads, feed_dim, num_layers, num_classes).to(device)

# 2개의 손실 생성
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
mlm_loss_fn = nn.CrossEntropyLoss(ignore_index=-100)  # MLM
sop_loss_fn = nn.CrossEntropyLoss() # SOP

[('일은 왜 해도 해도 끝이 없을까? 화가 난다.', '많이 힘드시겠어요. 주위에 의논할 상대가 있나요?', 0), ('많이 힘드시겠어요. 주위에 의논할 상대가 있나요?', '일은 왜 해도 해도 끝이 없을까? 화가 난다.', 1), ('이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.', '급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?', 0), ('급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?', '이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.', 1), ('회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스트레스 받아. ', '회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 하면 좋을까요?', 0), ('회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 하면 좋을까요?', '회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스트레스 받아. ', 1), ('직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 섭섭해.', '관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있게 도움을 줄까요?', 0), ('관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있게 도움을 줄까요?', '직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 섭섭해.', 1), ('얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.', '무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는 것일까요?', 0), ('무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는 것일까요?', '얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무

In [ ]:
import os

if not os.path.isdir('./checkpoint'):
    os.mkdir('./checkpoint')

train_loss = []
test_acc = []
# 학습 루프
for epoch in range(num_epochs):
    t_mlm_loss = 0.0
    t_sop_loss = 0.0
    model.train()
    for i, (inp, labels, mask) in enumerate(train_loader):
        inp = inp.long().to(device)
        nsp_label = labels.to(device)
        mask = mask.to(device)
        mlm_inp, mlm_labels = mask_tokens_for_mlm(inp.long(), special_tokens_ids, mask_id, vocab_size)
        mlm_logits, sop_logits = model(mlm_inp, mask)

        # MLM은 batch*seq 형상 변환 하여 손실 계산
        mlm_loss = mlm_loss_fn(mlm_logits.view(-1, vocab_size), mlm_labels.view(-1))
        # SOP 손실 계산
        sop_loss = sop_loss_fn(sop_logits, nsp_label)

        # 두 손실을 더하여 역전파
        loss = mlm_loss + sop_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        t_mlm_loss += mlm_loss.item()
        t_sop_loss += sop_loss.item()

        if (i+1) % 200 == 0:
            print(f"Epoch: {epoch}, Batch: {i+1}, MLM Loss: {t_mlm_loss/(i+1)}, SOP Loss:{t_sop_loss/(i+1)}")

    avg_loss = (t_mlm_loss + t_sop_loss) / len(train_loader)
    print(f"Train ===> Epoch {epoch+1} Loss: {avg_loss}")
    train_loss.append(avg_loss)
    checkpoint_path = f"checkpoint/ckpt{epoch}.pt"
    torch.save(model.state_dict(), checkpoint_path)

    # 평가 과정(NSP 정확도만 확인)
    v_acc = 0.0
    v_loss = 0.0
    model.eval()
    with torch.no_grad():
        for i, (inp, labels, mask) in enumerate(val_loader):
            inp = inp.long().to(device)
            labels = labels.to(device)
            mask = mask.to(device)
            mlm_logits, sop_logits = model(inp, mask)

            # SOP 정확도 연산
            acc = (sop_logits.argmax(dim=1) == labels).sum() / len(labels)
            v_acc += acc

        avg_acc = v_acc / len(val_loader)
        print(f"Val ===> Epoch {epoch+1}, Val_SOP_Accuracy: {avg_acc}")
        test_acc.append(avg_acc.cpu().numpy())

    print('-'*30)
torch.save(model, "model1.pt")

Epoch: 0, Batch: 200, MLM Loss: 7.252193458080292, SOP Loss:0.6714782665669918
Epoch: 0, Batch: 400, MLM Loss: 6.885182340145111, SOP Loss:0.419650372969918
Epoch: 0, Batch: 600, MLM Loss: 6.716233363946279, SOP Loss:0.3068597737917056
Epoch: 0, Batch: 800, MLM Loss: 6.620505163669586, SOP Loss:0.24641353070852345
Epoch: 0, Batch: 1000, MLM Loss: 6.559554091930389, SOP Loss:0.20783814616780727
Epoch: 0, Batch: 1200, MLM Loss: 6.511069982846578, SOP Loss:0.18100157124844068
Train ===> Epoch 1 Loss: 6.6639475409997635
Val ===> Epoch 1, Val_SOP_Accuracy: 0.99651700258255
------------------------------
Epoch: 1, Batch: 200, MLM Loss: 6.227266519069672, SOP Loss:0.046208606362342836
Epoch: 1, Batch: 400, MLM Loss: 6.213429446220398, SOP Loss:0.042574848461372314
Epoch: 1, Batch: 600, MLM Loss: 6.190332304636637, SOP Loss:0.04284097826671011
Epoch: 1, Batch: 800, MLM Loss: 6.17110288143158, SOP Loss:0.04103899515830563
Epoch: 1, Batch: 1000, MLM Loss: 6.152517322063446, SOP Loss:0.0403807757

BERT vs ALBERT

| 구분          | **BERT (실습)**                            | **ALBERT (실습)**                                      |
| ----------- | ---------------------------------------- | ---------------------------------------------------- |
| **임베딩 구조**  | `Embedding(vocab, 128)` → 바로 Transformer | `Embedding(vocab, 64)` → `Linear(64→128)` projection |
| **레이어 구조**  | `MTBlock(128)` × **4개 (각자 다른 가중치)**      | `MTBlock(128)` × **1개를 4번 공유**                       |
| **파라미터 규모** | 큼                                        | 작음 (임베딩 절반 + block 공유)                               |
| **속도**      | 상대적으로 느림                                 | 더 빠르고 가벼움                                            |





- 가중치/메모리/성능 비교

| 항목        | BERT          | ALBERT           |
| --------- | ------------- | ---------------- |
| layer 가중치 | 모든 block이 독립적 | 하나의 block을 4번 반복 |
| 메모리       | 큼             | 매우 작음            |
| 성능        | 안정적           | 가벼우며 빠름          |
